In [1]:
import pandas as pd
import numpy as np

# Читаем данные

In [2]:
X_train_in = pd.read_csv("X_train.csv", index_col=0)
y_train_in = pd.read_csv("y_train.csv",index_col=0)
X_val = pd.read_csv("X_test.csv", index_col=0)
info = pd.read_csv("studs_info.csv").drop_duplicates(subset=["STD_ID"])

In [3]:
X_train_in

,STD_ID,НАПРАВЛЕНИЕ,ГОД,КУРС,СЕМЕСТР,АТТЕСТАЦИЯ,ДИСЦИПЛИНА
51081,175711,38.03.02 Менеджмент,2018-2019,2.0,4.0,Экзамен,Маркетинг
72921,100647,31.05.01 Лечебное дело,2015-2016,3.0,5.0,Экзамен,"Пропедевтика внутренних болезней, лучевая диаг..."
96438,199311,31.05.03 Стоматология,2019-2020,1.0,1.0,Экзамен,Химия
273044,162002,38.03.02 Менеджмент,2017-2018,2.0,3.0,Экзамен,Статистика
371309,108384,31.05.01 Лечебное дело,2018-2019,5.0,9.0,Экзамен,Педиатрия
...,...,...,...,...,...,...,...
224856,176141,40.03.01 Юриспруденция,2017-2018,1.0,2.0,Курсовая работа,Конституционное право
260998,182794,31.05.01 Лечебное дело,2018-2019,2.0,4.0,Экзамен,Нормальная физиология
57245,162391,31.05.01 Лечебное дело,2016-2017,1.0,2.0,Дифференцированный зачет,Иностранный язык
218529,202479,38.03.01 Экономика,2017-2018,1.0,1.0,Дифференцированный зачет,Информатика


In [4]:
X_train_in = X_train_in.reset_index().drop(columns=["index"])
X_train_in = pd.merge(X_train_in.reset_index(), info).sort_values(by="index")

X_val = X_val.reset_index().drop(columns=["index"])
X_val = pd.merge(X_val.reset_index(), info).sort_values(by="index")

X_train_in.index = pd.read_csv("X_train.csv", index_col=0).index
X_val.index = pd.read_csv("X_test.csv", index_col=0).index
X_val = X_val.drop(columns=["index"])
X_train_in = X_train_in.drop(columns=["index"])

In [5]:
X_train_in['НАПРАВЛЕНИЕ_clustered'] = X_train_in['НАПРАВЛЕНИЕ'].agg([lambda x: x[:2]])
X_val['НАПРАВЛЕНИЕ_clustered'] = X_val['НАПРАВЛЕНИЕ'].agg([lambda x: x[:2]])

# Все виды определенных колонок

In [6]:
att_types = list(set(list(X_train_in["АТТЕСТАЦИЯ"].values) + list(X_val["АТТЕСТАЦИЯ"].values)))
disc_types = list(set(list(X_train_in["ДИСЦИПЛИНА"].values) + list(X_val["ДИСЦИПЛИНА"].values)))
napr_types = list(set(list(X_train_in["НАПРАВЛЕНИЕ"].values) + list(X_val["НАПРАВЛЕНИЕ"].values)))
napr_c_types = list(set(list(X_train_in["НАПРАВЛЕНИЕ_clustered"].values) + list(X_val["НАПРАВЛЕНИЕ_clustered"].values)))
school = list(set(list(X_train_in["Что именно закончил"].values) + list(X_val["Что именно закончил"].values)))

# Преобразовываем длинные название учереждений с предыдущим образованием в определенный тип - лицей/школа/университет

In [7]:
sch = ["сош", "школа", "осш", "ош", "коррекционная"]
gym = ['гимназия']
tech = ["техникум", "лицей", "колледж", "училище", "пу", "пту"]
uni = ["университет", "академия"]

In [8]:
schs = []
gyms = []
techs = []
unis = []
for i in school:    
    for j in sch:
        if j in str(i).lower():
            schs.append(i)
    
    for j in gym:
        if j in str(i).lower():
            gyms.append(i)
    
    for j in tech:
        if j in str(i).lower():
            techs.append(i)    
    for j in uni:
        if j in str(i).lower():
            unis.append(i)

In [9]:
X_train_in["mark"] = y_train_in["mark"]

In [10]:
def track(x):
    d = [schs,gyms,techs,unis]
    a = ["Школа", "Гимназия", "СП", "Университет"]
    for i in range(len(d)):
        if x in d[i]:
            return a[i]
    return "Школа"

In [11]:
X_train_in["Что именно закончил"] = X_train_in["Что именно закончил"].agg([track])
X_val["Что именно закончил"] = X_val["Что именно закончил"].agg([track])

# Создаем уникальные парные столбцы на основе существующих колонок

In [12]:
X_train_in["АТДИ"] = X_train_in["АТТЕСТАЦИЯ"] + X_train_in["ДИСЦИПЛИНА"]
X_val["АТДИ"] = X_val["АТТЕСТАЦИЯ"] + X_val["ДИСЦИПЛИНА"]

X_train_in["КУНА"] = X_train_in["КУРС"].astype("str") + X_train_in["НАПРАВЛЕНИЕ"]
X_val["КУНА"] = X_val["КУРС"].astype("str") + X_val["НАПРАВЛЕНИЕ"]

X_train_in["КаНА"] = X_train_in["Категория обучения"].astype("str") + X_train_in["НАПРАВЛЕНИЕ"]
X_val["КаНА"] = X_val["Категория обучения"].astype("str") + X_val["НАПРАВЛЕНИЕ"]

X_train_in["СЕНА"] = X_train_in["СЕМЕСТР"].astype("str") + X_train_in["НАПРАВЛЕНИЕ"]
X_val["СЕНА"] = X_val["СЕМЕСТР"].astype("str") + X_val["НАПРАВЛЕНИЕ"]

X_train_in["ПоНА"] = X_train_in["Пол"].astype("str") + X_train_in["НАПРАВЛЕНИЕ"]
X_val["ПоНА"] = X_val["Пол"].astype("str") + X_val["НАПРАВЛЕНИЕ"]

X_train_in["ПоАТ"] = X_train_in["Пол"].astype("str") + X_train_in["АТТЕСТАЦИЯ"]
X_val["ПоАТ"] = X_val["Пол"].astype("str") + X_val["АТТЕСТАЦИЯ"]

X_train_in["ПоДИ"] = X_train_in["Пол"].astype("str") + X_train_in["ДИСЦИПЛИНА"]
X_val["ПоДИ"] = X_val["Пол"].astype("str") + X_val["ДИСЦИПЛИНА"]

X_train_in["ПоДИАТ"] = X_train_in["Пол"].astype("str") + X_train_in["АТДИ"]
X_val["ПоДИАТ"] = X_val["Пол"].astype("str") + X_val["АТДИ"]

# mean-энкодинг столбцов - средняя оценка по студентам удовлетворяющим условиям

In [13]:
def add_feat_mean(cat):
    mean_mark = X_train_in.groupby(cat).mean()["mark"]
    X_train_in.loc[:,f"{cat}_mean"] = X_train_in.loc[:,cat].map(mean_mark)
    X_val.loc[:,f"{cat}_mean"] = X_val.loc[:,cat].map(mean_mark)
    
add_feat_mean("STD_ID")
add_feat_mean("ДИСЦИПЛИНА")
add_feat_mean("АТТЕСТАЦИЯ")
add_feat_mean("НАПРАВЛЕНИЕ")
add_feat_mean("НАПРАВЛЕНИЕ_clustered")
add_feat_mean("ГОД")
add_feat_mean("КУРС")
add_feat_mean("СЕМЕСТР")
add_feat_mean("Категория обучения")
add_feat_mean("Пол")
add_feat_mean("Что именно закончил")
add_feat_mean("АТДИ")
add_feat_mean("КУНА")
add_feat_mean("КаНА")
add_feat_mean("СЕНА")
add_feat_mean("ПоНА")
add_feat_mean("ПоАТ")
add_feat_mean("ПоДИ")
add_feat_mean("ПоДИАТ")

# Сохранение полученных фич

In [14]:
X_train_in.to_csv("X_train_app.csv")
X_val.to_csv("X_val_app.csv")